In [0]:
# Mount Google Drive if working on Colab

from google.colab import drive
drive.mount("/drive")

In [0]:
# Install Pytesseract and Image Super Resolition (ISR) Library 

!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install ISR

In [0]:
# Import required packages

import cv2
import numpy as np
import matplotlib.pyplot as plt
import re
from PIL import Image
import pytesseract as ocr
from ISR.models import RRDN

In [0]:
# Load Pretrained ESRGAN Model

SR_Model = RRDN(weights='gans')

In [0]:
# Verhoeff Algorithm

multiplication_table = (
    (0, 1, 2, 3, 4, 5, 6, 7, 8, 9),
    (1, 2, 3, 4, 0, 6, 7, 8, 9, 5),
    (2, 3, 4, 0, 1, 7, 8, 9, 5, 6),
    (3, 4, 0, 1, 2, 8, 9, 5, 6, 7),
    (4, 0, 1, 2, 3, 9, 5, 6, 7, 8),
    (5, 9, 8, 7, 6, 0, 4, 3, 2, 1),
    (6, 5, 9, 8, 7, 1, 0, 4, 3, 2),
    (7, 6, 5, 9, 8, 2, 1, 0, 4, 3),
    (8, 7, 6, 5, 9, 3, 2, 1, 0, 4),
    (9, 8, 7, 6, 5, 4, 3, 2, 1, 0))

permutation_table = (
    (0, 1, 2, 3, 4, 5, 6, 7, 8, 9),
    (1, 5, 7, 6, 2, 8, 3, 0, 9, 4),
    (5, 8, 0, 3, 7, 9, 6, 1, 4, 2),
    (8, 9, 1, 6, 0, 4, 3, 5, 2, 7),
    (9, 4, 5, 3, 1, 2, 6, 8, 7, 0),
    (4, 2, 8, 6, 5, 7, 3, 9, 0, 1),
    (2, 7, 9, 3, 8, 0, 6, 4, 1, 5),
    (7, 0, 4, 6, 9, 1, 3, 2, 5, 8))

def compute_checksum(number):
    
    """Calculate the Verhoeff checksum over the provided number. The checksum
    is returned as an int. Valid numbers should have a checksum of 0."""
    
    # transform number list
    number = tuple(int(n) for n in reversed(str(number)))
    
    # calculate checksum
    checksum = 0
    
    for i, n in enumerate(number):
        checksum = multiplication_table[checksum][permutation_table[i % 8][n]]
    
    return checksum

In [0]:
# Path to Input Image

input_path = "/drive/My Drive/Flipr/data/1.png"

In [0]:
# Load Image and Convert to Grayscale

img = Image.open(input_path).convert('LA')
img.save('grayscale.png')

In [0]:
# OCR using Pytesseract

settings = ('-l eng --oem 3 --psm 11')

result = ocr.image_to_string(Image.open('grayscale.png'),config=settings)
bounding_boxes = ocr.image_to_boxes(Image.open('grayscale.png'),config=settings).split(" 0\n")

print(result)
bounding_boxes

In [0]:
# Search Possible UIDs with Bounding Boxes

start_char_pos = 0

lines = result.split('\n')
possible_UIDs = []

for line in lines:
  
  words = line.split()

  if(len(words)==0):
    continue

  Combined = "".join(words)

  UID = re.search("[0-9]{12}",Combined)    

  if UID!=None:

    # Check Validity

    if compute_checksum(int(UID.group())) == 0:
      possible_UIDs.append([int(UID.group()),start_char_pos + UID.start()])

  start_char_pos += len(Combined)
  print(Combined,start_char_pos)

# Get Unique Possible UIDs

possible_UIDs = np.unique(possible_UIDs,axis=0)

In [47]:
# Print Results

print(possible_UIDs[:,0])

[704809741779]


In [0]:
# Mask found UIDs

img = Image.open(input_path)
pixels = img.load()

for UID in possible_UIDs:

  top_corner = bounding_boxes[UID[1]].split()
  bottom_corner = bounding_boxes[UID[1] + 7].split()

  for x in range(int(top_corner[1]),int(bottom_corner[3])+1):
      for y in range(img.height-int(top_corner[4]),img.height-int(bottom_corner[2])+1):
          pixels[x, y] = (0,0,0)

img.save("masked_img.png")

In [0]:
# Show Original and Masked Images

plt.figure(figsize=(10,8))
plt.subplot(2,1,1)
plt.title("Original")
plt.imshow(Image.open(input_path))

plt.subplot(2,1,2)
plt.title("Masked")
plt.imshow(Image.open("masked_img.png"))

plt.show()